**COVID - Drug Discovery for COVID19**

The objective is to prepare a machine learning model that can be used to propose potential novel effective drugs to fight SARS-CoV-2, the virus responsible for COVID-19.

You are provided with a dataset containing drug molecules (encoded as SMILES) and their binding affinities. The task is to use this dataset to make a regression model for binding affinity prediction.

**About Dataset**


SARS-CoV-2 virus contains proteins responsible for action and replication of the virus. The protein functions can be stopped by introducing drug molecules that are capable of blocking the protein. In other words, preparation of a drug involves finding molecules that can effectively bind to the protein i.e have a high binding affinity.
In this task, you are provided with a dataset of drug molecules and their binding affinity towards SARS-CoronaVirus Main Proteaese(Mpro), one of the proteins in the target virus.
The data has been generated using Protein-Ligand docking.

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## ** To install rdkit use the below code**

In [9]:
import sys
import os
import requests
import subprocess
import shutil
from logging import getLogger, StreamHandler, INFO


logger = getLogger(__name__)
logger.addHandler(StreamHandler())
logger.setLevel(INFO)


def install(
        chunk_size=4096,
        file_name="Miniconda3-latest-Linux-x86_64.sh",
        url_base="https://repo.continuum.io/miniconda/",
        conda_path=os.path.expanduser(os.path.join("~", "miniconda")),
        rdkit_version=None,
        add_python_path=True,
        force=False):
    """install rdkit from miniconda
    ```
    import rdkit_installer
    rdkit_installer.install()
    ```
    """

    python_path = os.path.join(
        conda_path,
        "lib",
        "python{0}.{1}".format(*sys.version_info),
        "site-packages",
    )

    if add_python_path and python_path not in sys.path:
        logger.info("add {} to PYTHONPATH".format(python_path))
        sys.path.append(python_path)

    if os.path.isdir(os.path.join(python_path, "rdkit")):
        logger.info("rdkit is already installed")
        if not force:
            return

        logger.info("force re-install")

    url = url_base + file_name
    python_version = "{0}.{1}.{2}".format(*sys.version_info)

    logger.info("python version: {}".format(python_version))

    if os.path.isdir(conda_path):
        logger.warning("remove current miniconda")
        shutil.rmtree(conda_path)
    elif os.path.isfile(conda_path):
        logger.warning("remove {}".format(conda_path))
        os.remove(conda_path)

    logger.info('fetching installer from {}'.format(url))
    res = requests.get(url, stream=True)
    res.raise_for_status()
    with open(file_name, 'wb') as f:
        for chunk in res.iter_content(chunk_size):
            f.write(chunk)
    logger.info('done')

    logger.info('installing miniconda to {}'.format(conda_path))
    subprocess.check_call(["bash", file_name, "-b", "-p", conda_path])
    logger.info('done')

    logger.info("installing rdkit")
    subprocess.check_call([
        os.path.join(conda_path, "bin", "conda"),
        "install",
        "--yes",
        "-c", "rdkit",
        "python=={}".format(python_version),
        "rdkit" if rdkit_version is None else "rdkit=={}".format(rdkit_version)])
    logger.info("done")

    import rdkit
    logger.info("rdkit-{} installation finished!".format(rdkit.__version__))


if __name__ == "__main__":
    install()

rdkit is already installed
rdkit is already installed


**Installing a package for mol2vec**

In [4]:
!pip install git+https://github.com/samoturk/mol2vec;

  Cloning https://github.com/samoturk/mol2vec to /tmp/pip-req-build-8uxng7x9
  Running command git clone -q https://github.com/samoturk/mol2vec /tmp/pip-req-build-8uxng7x9
  Created wheel for mol2vec: filename=mol2vec-0.1-cp36-none-any.whl size=14026 sha256=5f0c63de6bb59edcde3ae8e0dcae4544d177eef03b9ea0796e0b637ef15de9c5
  Stored in directory: /tmp/pip-ephem-wheel-cache-l_ijpz99/wheels/96/0f/2d/a1092b9677c96453dc244b209544cac61bc8b974cbffb50063
Successfully built mol2vec


In [0]:
#import the essential libraries


%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
train =  pd.read_csv('/content/drive/My Drive/covid/train.csv')

In [12]:
train.head()

,SMILES sequence,Binding Affinity
0,CCNC(C)C(NC)c1ccccc1,-18.0861
1,CONC(=O)c1cncnc1,-17.5783
2,CCNC1CCCN(Cc2ccsc2)C1,-20.3645
3,CC(NC(=O)CSCCN)c1ccccc1,-19.3144
4,CCC(CS)CN(C)c1ccccc1,-15.8451


In [0]:

train_aff = train['Binding Affinity']
train.drop(columns='Binding Affinity',inplace=True)

In [14]:
train.head()

,SMILES sequence
0,CCNC(C)C(NC)c1ccccc1
1,CONC(=O)c1cncnc1
2,CCNC1CCCN(Cc2ccsc2)C1
3,CC(NC(=O)CSCCN)c1ccccc1
4,CCC(CS)CN(C)c1ccccc1


In [15]:
#It stores the binding affinity
train_aff.head()

0   -18.0861
1   -17.5783
2   -20.3645
3   -19.3144
4   -15.8451
Name: Binding Affinity, dtype: float64

In [0]:
test = pd.read_csv('/content/drive/My Drive/covid/test.csv')
test_name = pd.read_csv('/content/drive/My Drive/covid/test.csv')## Creating this just as a copy of test data to create the csv using this later

In [17]:
test.head()

,SMILES sequence,Binding Affinity
0,Cc1ccc(C2CNCCN2C)cc1,NaN
1,CCOC(CO)c1ccccc1,NaN
2,CC(=O)Nc1cnn(C)n1,NaN
3,CCC(C)NCc1ncccn1,NaN
4,CC(C)=C1CC(N)C1,NaN


In [18]:

test_name.head()

,SMILES sequence,Binding Affinity
0,Cc1ccc(C2CNCCN2C)cc1,NaN
1,CCOC(CO)c1ccccc1,NaN
2,CC(=O)Nc1cnn(C)n1,NaN
3,CCC(C)NCc1ncccn1,NaN
4,CC(C)=C1CC(N)C1,NaN


In [0]:
test.drop(columns='Binding Affinity',inplace=True)
test_name.drop(columns='Binding Affinity',inplace=True)

In [20]:
test.head()

,SMILES sequence
0,Cc1ccc(C2CNCCN2C)cc1
1,CCOC(CO)c1ccccc1
2,CC(=O)Nc1cnn(C)n1
3,CCC(C)NCc1ncccn1
4,CC(C)=C1CC(N)C1


In [21]:
test_name.head()

,SMILES sequence
0,Cc1ccc(C2CNCCN2C)cc1
1,CCOC(CO)c1ccccc1
2,CC(=O)Nc1cnn(C)n1
3,CCC(C)NCc1ncccn1
4,CC(C)=C1CC(N)C1


In [0]:
##This is the sample of submission (Submission must be in this format)
sample =  pd.read_csv('/content/drive/My Drive/covid/sample_submission.csv')

In [24]:
sample.head()

,SMILES sequence,Binding Affinity
0,Cc1ccc(C2CNCCN2C)cc1,-10.0
1,CCOC(CO)c1ccccc1,-10.0
2,CC(=O)Nc1cnn(C)n1,-10.0
3,CCC(C)NCc1ncccn1,-10.0
4,CC(C)=C1CC(N)C1,-10.0


In [0]:
from rdkit import Chem 


In [0]:
train['mol'] = train['SMILES sequence'].apply(lambda x: Chem.MolFromSmiles(x)) 
test['mol'] = test['SMILES sequence'].apply(lambda x: Chem.MolFromSmiles(x)) 

In [27]:
#Now let's see what we've got
print(type(train['mol'][0]))

<class 'rdkit.Chem.rdchem.Mol'>


In [28]:
print(type(test['mol'][0]))

<class 'rdkit.Chem.rdchem.Mol'>


In [32]:
from rdkit.Chem import Draw
mols = train['mol'][:2]

#MolsToGridImage allows to paint a number of molecules at a time
Draw.MolsToGridImage(mols, molsPerRow=5, useSVG=True, legends=list(train['SMILES sequence'][:20].values))

'<?xml version=\'1.0\' encoding=\'iso-8859-1\'?>\n<svg version=\'1.1\' baseProfile=\'full\'\n              xmlns=\'http://www.w3.org/2000/svg\'\n                      xmlns:rdkit=\'http://www.rdkit.org/xml\'\n                      xmlns:xlink=\'http://www.w3.org/1999/xlink\'\n                  xml:space=\'preserve\'\nwidth=\'1000px\' height=\'200px\' viewBox=\'0 0 1000 200\'>\n<!-- END OF HEADER -->\n<rect style=\'opacity:1.0;fill:#FFFFFF;stroke:none\' width=\'1000\' height=\'200\' x=\'0\' y=\'0\'> </rect>\n<rect style=\'opacity:1.0;fill:#FFFFFF;stroke:none\' width=\'1000\' height=\'200\' x=\'0\' y=\'0\'> </rect>\n<rect style=\'opacity:1.0;fill:#FFFFFF;stroke:none\' width=\'1000\' height=\'200\' x=\'0\' y=\'0\'> </rect>\n<path class=\'bond-0\' d=\'M 41.2474,24.6306 L 43.0908,58.4829\' style=\'fill:none;fill-rule:evenodd;stroke:#000000;stroke-width:2px;stroke-linecap:butt;stroke-linejoin:miter;stroke-opacity:1\' />\n<path class=\'bond-1\' d=\'M 43.0908,58.4829 L 55.7613,64.9063\' style=

![alt text](https://drive.google.com/uc?id=1exJGBWzAM7L5cN9oAzFAVPhuw9Uv3Wja)

In [33]:
train.head()

,SMILES sequence,mol
0,CCNC(C)C(NC)c1ccccc1,<rdkit.Chem.rdchem.Mol object at 0x7f04d05cfcb0>
1,CONC(=O)c1cncnc1,<rdkit.Chem.rdchem.Mol object at 0x7f04d05cfd00>
2,CCNC1CCCN(Cc2ccsc2)C1,<rdkit.Chem.rdchem.Mol object at 0x7f04d05cfd50>
3,CC(NC(=O)CSCCN)c1ccccc1,<rdkit.Chem.rdchem.Mol object at 0x7f04d05cfda0>
4,CCC(CS)CN(C)c1ccccc1,<rdkit.Chem.rdchem.Mol object at 0x7f04d05cfdf0>


In [34]:
test.head()

,SMILES sequence,mol
0,Cc1ccc(C2CNCCN2C)cc1,<rdkit.Chem.rdchem.Mol object at 0x7f04cc32cc10>
1,CCOC(CO)c1ccccc1,<rdkit.Chem.rdchem.Mol object at 0x7f04cc32cee0>
2,CC(=O)Nc1cnn(C)n1,<rdkit.Chem.rdchem.Mol object at 0x7f04d0fd7800>
3,CCC(C)NCc1ncccn1,<rdkit.Chem.rdchem.Mol object at 0x7f04d0fd7710>
4,CC(C)=C1CC(N)C1,<rdkit.Chem.rdchem.Mol object at 0x7f04cc330030>


In [35]:


#Now we'll load a pre-trained mol2vec model. It's trained with radius=1 for Morgan fingerprints to yield 300 dimensional embeddings.

#Loading pre-trained model via word2vec
from gensim.models import word2vec
model = word2vec.Word2Vec.load('/content/drive/My Drive/model_300dim.pkl')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
from mol2vec.features import mol2alt_sentence, mol2sentence, MolSentence, DfVec, sentences2vec
from gensim.models import word2vec

In [37]:
print('Molecular sentence:', mol2alt_sentence(train['mol'][1], radius=1))
print('\nMolSentence object:', MolSentence(mol2alt_sentence(train['mol'][1], radius=1)))
print('\nDfVec object:',DfVec(sentences2vec(MolSentence(mol2alt_sentence(train['mol'][1], radius=1)), model, unseen='UNK')))

Molecular sentence: ['2246728737', '3975275337', '864674487', '903112553', '847961216', '2204949651', '2246699815', '1054767590', '864942730', '1510328189', '3217380708', '2994748777', '3218693969', '3777168895', '2041434490', '3118255683', '3218693969', '725322217', '2041434490', '3118255683', '3218693969', '3777168895']

MolSentence object: MolSentence with 22 words

DfVec object: (22, 300) dimensional vector


In [38]:
print('Molecular sentence:', mol2alt_sentence(test['mol'][1], radius=1))
print('\nMolSentence object:', MolSentence(mol2alt_sentence(test['mol'][1], radius=1)))
print('\nDfVec object:',DfVec(sentences2vec(MolSentence(mol2alt_sentence(test['mol'][1], radius=1)), model, unseen='UNK')))

Molecular sentence: ['2246728737', '3542456614', '2245384272', '3994088662', '864674487', '2814583100', '2245273601', '602889953', '2245384272', '4022716898', '864662311', '1535166686', '3217380708', '3579962709', '3218693969', '951226070', '3218693969', '98513984', '3218693969', '98513984', '3218693969', '98513984', '3218693969', '951226070']

MolSentence object: MolSentence with 24 words

DfVec object: (24, 300) dimensional vector


In [0]:
#Constructing sentences
train['sentence'] = train.apply(lambda x: MolSentence(mol2alt_sentence(x['mol'], 1)), axis=1)


In [0]:
test['sentence'] = test.apply(lambda x: MolSentence(mol2alt_sentence(x['mol'], 1)), axis=1)

In [41]:
#Extracting embeddings to a numpy.array
#Note that we always should mark unseen='UNK' in sentence2vec() so that model is taught how to handle unknown substructures
train['mol2vec'] = [DfVec(x) for x in sentences2vec(train['sentence'], model, unseen='UNK')]
X = np.array([x.vec for x in train['mol2vec']])
y = train_aff.values

X.shape

(9000, 300)

In [42]:
test['mol2vec'] = [DfVec(x) for x in sentences2vec(test['sentence'], model, unseen='UNK')]
test_value= np.array([x.vec for x in test['mol2vec']])
test_value.shape

(2500, 300)

We split the data into train and validate and analyze with various different regression algorithms like Ridge,svm,lasso,etc and also with their different parameters

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.1, random_state=1)

In [0]:
from sklearn import svm

After alot of testing we have found out that at C=100 our data works best

Testing with SVM

In [0]:
clf = svm.SVR(C=100)

In [46]:
clf.fit(X_train, y_train)


SVR(C=110, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [0]:
y_pred=clf.predict(X_test)

In [50]:
    from sklearn.metrics import mean_squared_error
    from math import sqrt
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    print(rmse)

2.328732546851693


In [0]:
from sklearn.metrics import mean_absolute_error

In [52]:
mean_absolute_error(y_test, y_pred)

1.6724466865753191

Testing with Ridge

In [0]:
from sklearn.linear_model import RidgeCV
ridge = RidgeCV()

In [57]:
ridge.fit(X_train, y_train)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]), cv=None, fit_intercept=True,
        gcv_mode=None, normalize=False, scoring=None, store_cv_values=False)

In [0]:
y_pred_r = ridge.predict(X_test)

In [59]:
rmse = sqrt(mean_squared_error(y_test, y_pred_r))
rmse

2.37664859146776

In [60]:
print(mean_absolute_error(y_test, y_pred_r))

1.7649498076453982


We have found out that of all Svm.SVR works best of sll the algorithm with 'C' value of 100

In [0]:
clf1 = svm.SVR(C=100)


In [63]:
clf1.fit(X, y)

SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [0]:
y_pred_svm=clf.predict(test_value)

In [65]:
 final_aff=y_pred_svm.tolist()
 type(final_aff)

list

In [0]:
 final_test= test_name.values.tolist()

In [67]:
type(final_test)

list

In [71]:
print(final_test)

[['Cc1ccc(C2CNCCN2C)cc1'], ['CCOC(CO)c1ccccc1'], ['CC(=O)Nc1cnn(C)n1'], ['CCC(C)NCc1ncccn1'], ['CC(C)=C1CC(N)C1'], ['CN1CCCc2cc(CON)ccc21'], ['CC12CNCC1CN(CC(N)=O)C2'], ['CCNC(C)C(C)c1cnccc1C'], ['CCN(Cc1ccccc1)C(C)CCCNC'], ['CCC(=O)c1cc(C)ccn1'], ['C=CC(O)c1cc(C)ccn1'], ['C#CCCOc1cnccc1C'], ['Cn1nc2ccccc2c1S(N)(=O)=O'], ['Cn1cnn(CC(N)=O)c1=O'], ['CC(NCCSc1ccccc1)c1ccncc1'], ['Cc1ccsc1-c1ccc(O)nc1'], ['N#Cc1ncccc1N1CC2CC1CN2'], ['N#Cc1cnccc1SCC(N)=O'], ['N#Cc1ccc(C2NCCCCC2=O)cn1'], ['CC(C)C(C)Sc1ccc(C#N)cn1'], ['Cc1ccnc(C=CCCN)c1'], ['CCOC(CC)C(=O)c1cnccc1C'], ['Cc1ccncc1C(O)CNCC(C)C'], ['CS(=O)(=O)c1ncc(N)cn1'], ['OCC(O)CCSCc1ccccc1'], ['COC(=O)CNCc1cc(C)ccn1'], ['CCOC(c1ccccc1)C(CC)NN'], ['Cc1ccncc1C(=O)CCCN(C)C'], ['C=CCCSCCNc1cc(C)ccn1'], ['CCNC(=S)NNC(=O)Cc1ccccc1'], ['OC(CCCc1ccccc1)c1cccnc1'], ['CC(=O)CC(C)c1cnccc1C'], ['CN1CCC(O)(c2ccoc2)CC1'], ['Cc1ccnc(NC(=O)C#CCN)c1'], ['N#Cc1cnccc1NCCCO'], ['CCSCc1cncc(C#N)c1'], ['NC1=CCOC1=O'], ['CNC(CSC1CCCCC1)Cc1cccnc1'], ['COC(=O)c1ccc(

In [0]:
# The final_list which contains the name of chemical compounds in SMILES format is being converted from 2D to 1D.
from itertools import chain 
final_name = list(chain.from_iterable(final_test)) 

In [72]:
print(final_name)

['Cc1ccc(C2CNCCN2C)cc1', 'CCOC(CO)c1ccccc1', 'CC(=O)Nc1cnn(C)n1', 'CCC(C)NCc1ncccn1', 'CC(C)=C1CC(N)C1', 'CN1CCCc2cc(CON)ccc21', 'CC12CNCC1CN(CC(N)=O)C2', 'CCNC(C)C(C)c1cnccc1C', 'CCN(Cc1ccccc1)C(C)CCCNC', 'CCC(=O)c1cc(C)ccn1', 'C=CC(O)c1cc(C)ccn1', 'C#CCCOc1cnccc1C', 'Cn1nc2ccccc2c1S(N)(=O)=O', 'Cn1cnn(CC(N)=O)c1=O', 'CC(NCCSc1ccccc1)c1ccncc1', 'Cc1ccsc1-c1ccc(O)nc1', 'N#Cc1ncccc1N1CC2CC1CN2', 'N#Cc1cnccc1SCC(N)=O', 'N#Cc1ccc(C2NCCCCC2=O)cn1', 'CC(C)C(C)Sc1ccc(C#N)cn1', 'Cc1ccnc(C=CCCN)c1', 'CCOC(CC)C(=O)c1cnccc1C', 'Cc1ccncc1C(O)CNCC(C)C', 'CS(=O)(=O)c1ncc(N)cn1', 'OCC(O)CCSCc1ccccc1', 'COC(=O)CNCc1cc(C)ccn1', 'CCOC(c1ccccc1)C(CC)NN', 'Cc1ccncc1C(=O)CCCN(C)C', 'C=CCCSCCNc1cc(C)ccn1', 'CCNC(=S)NNC(=O)Cc1ccccc1', 'OC(CCCc1ccccc1)c1cccnc1', 'CC(=O)CC(C)c1cnccc1C', 'CN1CCC(O)(c2ccoc2)CC1', 'Cc1ccnc(NC(=O)C#CCN)c1', 'N#Cc1cnccc1NCCCO', 'CCSCc1cncc(C#N)c1', 'NC1=CCOC1=O', 'CNC(CSC1CCCCC1)Cc1cccnc1', 'COC(=O)c1ccc(C(C)C=O)cc1', 'CC(=O)CC(O)c1cnccc1C', 'CCCNCc1ccccc1S(N)(=O)=O', 'N#Cc1nccnc1

In [0]:
# We join the sequence name and its predicted affinity and convert it into a dataframe
df = pd.DataFrame(list(zip(fi, final_aff )), columns =['SMILES sequence', 'Binding Affinity']) 

In [74]:
df.head()

,SMILES sequence,Binding Affinity
0,Cc1ccc(C2CNCCN2C)cc1,-21.682052
1,CCOC(CO)c1ccccc1,-14.222270
2,CC(=O)Nc1cnn(C)n1,-23.436068
3,CCC(C)NCc1ncccn1,-19.540014
4,CC(C)=C1CC(N)C1,-19.953755


In [0]:
#Convert the dataframe into a csv named 'sub1.csv'
file_name_csv= 'sub1.csv'
df.to_csv(file_name_csv,index=False)


**References**

https://www.kaggle.com/vladislavkisin/tutorial-ml-in-chemistry-research-rdkit-mol2vec/data  (All the essential pre-trained libraries could be found in this kernel)


https://medium.com/atomwise/machine-learning-for-drug-discovery-in-a-nutshell-part-ii-24f90d5963d9


https://www.oreilly.com/library/view/deep-learning-for/9781492039822/


**Dataset available in this challenge at AIcrowd**

https://www.aicrowd.com/challenges/covid-drug-discovery-for-covid19